In [11]:
!pip install pandas
!pip install keras
!pip install matplotlib 
!pip install sklearn

In [12]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DeepLearning_SourceCode5/spam.csv',encoding = "ISO-8859-1")

# Keeping only the neccessary columns
y = data['v1']
data = data['v2']

data = data.apply(lambda x: x.lower())
data = data.apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data.values)
X = tokenizer.texts_to_sequences(data.values)
#padding
X = pad_sequences(X)
embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())
#encode labels
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(y)
y = to_categorical(integer_encoded)

X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

117/117 - 95s - loss: 0.1742 - accuracy: 0.9432
58/58 - 6s - loss: 0.0683 - accuracy: 0.9821
0.06827694922685623
0.9820554852485657
['loss', 'accuracy']


In [13]:
#save model
model.save("/content/drive/MyDrive/Colab Notebooks/2ndmodel.h5")

In [14]:
#save model
from keras.models import load_model
model = load_model("/content/drive/MyDrive/Colab Notebooks/2ndmodel.h5")

In [15]:
#prediction text
text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing"]
#tokenize and encoding the text
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(text)
X1 = tokenizer.texts_to_sequences(text)
#adding padding
X2 = pad_sequences(X1)

In [16]:
#prediction
model.predict(X2)

array([[0.9413479 , 0.05865207]], dtype=float32)

In [ ]:
#Grid search cv for hyperparameter tunning
from keras.wrappers.scikit_learn import KerasClassifier
m = KerasClassifier(build_fn=createmodel ,verbose=0)
batch_size = [10,20,40,50,5]
epochs = [2,5,15,10,20,35]
param_grid = dict(batch_size=batch_size,epochs=epochs)
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=m, param_grid=param_grid)
grid_result= grid.fit(X_train,Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))